classifier[0] - мой классификатор

model_params['features'] = лист из фичей в модели

df - датафрейм

instance_idx - номер строки

In [ ]:
# explanation = shap.Explanation(
#     values=shap_values,
#     base_values=explainer.expected_value,
#     data=df[model_params['features']].values,
#     feature_names=model_params['features']
# )

# shap.plots.waterfall(explanation[0])

In [ ]:
explainer = shap.TreeExplainer(classifier[0])
shap_values = explainer.shap_values(df[model_params['features']])

def print_shap_explanation_detailed(shap_values, explainer, feature_names, data, instance_idx, threshold=0.001):

    shap_val = shap_values[instance_idx]
    base_value = explainer.expected_value
    actual_values = data.iloc[instance_idx] if hasattr(data, 'iloc') else data[instance_idx]
    
    prediction = base_value + shap_val.sum()
    
    feature_importance = sorted(zip(feature_names, shap_val, actual_values), 
                               key=lambda x: abs(x[1]), reverse=True)
    
    print("="*90)
    print(f"Для наблюдения #{instance_idx}")
    print("="*90)
    print(f"Базовое значение (средний прогноз): {base_value:.4f}")
    print(f"Фактический прогноз: {prediction:.4f}")
    print(f"Сумма SHAP значений: {shap_val.sum():.4f}")
    print("-"*90)
    
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    BOLD = '\033[1m'
    END = '\033[0m'
    
    strong_impact = [(f, v, a) for f, v, a in feature_importance if abs(v) > 0.1]
    medium_impact = [(f, v, a) for f, v, a in feature_importance if 0.01 <= abs(v) <= 0.1]
    weak_impact = [(f, v, a) for f, v, a in feature_importance if 0.001 <= abs(v) < 0.01]
    
    def print_feature_group(features, title, color):
        if features:
            print(f"\n{color}{BOLD}{title}:{END}")
            print(f"{'Признак':<30} {'Факт.знач':<15} {'Вклад':<12} {'Влияние':<10}")
            print("-"*70)
            for feature, shap_value, actual_value in features:
                if abs(shap_value) > threshold:
                    direction = "↑ УВЕЛИЧ" if shap_value > 0 else "↓ УМЕНЬШ"
                    impact_color = GREEN if shap_value > 0 else RED
                    
                    if isinstance(actual_value, float):
                        actual_str = f"{actual_value:.4f}"
                    else:
                        actual_str = str(actual_value)
                    
                    print(f"{feature:<30} {actual_str:<15} {impact_color}{shap_value:+.4f}{END} {direction:<10}")
    
    print_feature_group(strong_impact, "СИЛЬНОЕ ВЛИЯНИЕ (|вклад| > 0.1)", RED)
    print_feature_group(medium_impact, "СРЕДНЕЕ ВЛИЯНИЕ (0.01 ≤ |вклад| ≤ 0.1)", YELLOW) 
    print_feature_group(weak_impact, "СЛАБОЕ ВЛИЯНИЕ (0.001 ≤ |вклад| < 0.01)", BLUE)
    
    print("-"*90)
    

print_shap_explanation_detailed(shap_values, explainer, model_params['features'], 
                               df[model_params['features']], instance_idx=0)

#для строки 0 просто выводит значения

In [ ]:
def create_shap_summary_df(shap_values, explainer, feature_names, data, client_ids=None, threshold=0.1):

    base_value = explainer.expected_value
    predictions = base_value + np.sum(shap_values, axis=1)
    
    results = []
    
    for instance_idx in range(len(data)):
        shap_val = shap_values[instance_idx]
        actual_values = data.iloc[instance_idx] if hasattr(data, 'iloc') else data[instance_idx]
        
        row_data = {
            'client_id': client_ids[instance_idx] if client_ids is not None else instance_idx,
            'Фактический_прогноз': predictions[instance_idx],
            'Базовое_значение': base_value,
            'Сумма_SHAP_значений': shap_val.sum()
        }
        
        strong_impact_features = []
        for feature, shap_value, actual_value in zip(feature_names, shap_val, actual_values):
            if abs(shap_value) > threshold:
                feature_key = f"{feature}_вклад"
                feature_value_key = f"{feature}_значение"
                row_data[feature_key] = shap_value
                row_data[feature_value_key] = actual_value
                strong_impact_features.append(feature)
        
        row_data['Признаки_с_сильным_влиянием'] = ', '.join(strong_impact_features)
        results.append(row_data)
    
    summary_df = pd.DataFrame(results)
    
    base_columns = ['id', 'Фактический_прогноз', 'Базовое_значение', 'Сумма_SHAP_значений', 'Признаки_с_сильным_влиянием']
    feature_columns = [col for col in summary_df.columns if col not in base_columns]
    
    contribution_cols = [col for col in feature_columns if col.endswith('_вклад')]
    value_cols = [col for col in feature_columns if col.endswith('_значение')]
    
    contribution_cols.sort()
    value_cols.sort()
    
    final_columns = base_columns + contribution_cols + value_cols
    summary_df = summary_df[final_columns]
    
    return summary_df



summary_df = create_shap_summary_df(shap_values, explainer, model_params['features'], 
                                   df[model_params['features']], 
                                   client_ids=df.index)

#для всех значений выведет табл